In [25]:
from operator import itemgetter
import numpy as np
import pandas as pd
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'


file5 = open('201811291521.txt', 'r')
dataset = file5.read()
print('data read')
dataset = dataset.split('\n')
data = []
for i in range(0, len(dataset)):
    data.append(dataset[i].split('|'))
data = sorted(data, key=itemgetter(0))
data.pop(0)
print('data sorted')

headings = ['stime', 'etime', 'sip', 'sport', 'sipint', 'mac', 'osname', 'osversion', 'fingerprint', 'dip', 'dport', 'dipint', 'dstmac', 'rosname', 'rosversion', 'rfingerprint', 'protocol', 'pkts', 'bytes', 'rpkts', 'rbytes', 'dur', 'iflags', 'riflags', 'uflags', 'ruflags', 'entropy', 'rentropy', 'tos', 'rtos', 'application', 'vlanint', 'domain', 'endreason', 'hash']
print('data : '+str(len(data)))
# for item in data:
#     item[0] = datetime.strptime(item[0], '%Y-%m-%d %H:%M:%S.%f')
#     item[1] = datetime.strptime(item[1], '%Y-%m-%d %H:%M:%S.%f')
#     print(item[0].strftime('%m/%d/%Y'))

data = np.array(data)
df = pd.DataFrame(data)
df.columns = headings
print('dataset created')
print(df.head())

edited_df = df.drop(['stime','etime','sipint','mac','osname','osversion','fingerprint','dipint','dstmac','rosname','rosversion','rfingerprint','iflags','riflags','uflags','ruflags','entropy','rentropy','tos','rtos','application','vlanint','domain','hash','pkts','bytes','rpkts','rbytes','dur','endreason'],axis=1)
print(edited_df.head())
# Get one hot encoding of columns B
one_hot = pd.get_dummies(edited_df['protocol'])
# Drop column B as it is now encoded
edited_df = edited_df.drop('protocol',axis = 1)
headers = []
for i in one_hot.columns:
    headers.append('protocol_' + i)
# Join the encoded df

one_hot.columns = headers
# edited_df = edited_df.join(one_hot)

def correct_ip(s):
    o = ''
    if '.' in s:
        for part in s.split('.'):
            part = part.zfill(3)
            o += part 
    else:
        o = o.zfill(12)
    o = o[:3] + '.' + o[3:]
    o = o[:7] + '.' + o[7:]
    o = o[:11] + '.' + o[11:]
    return o

def correct_port(s):
    return(s.zfill(5))
        


sip_headers = []
dip_headers = []

for i in range(4):
    sip_headers.append('sip_'+str(i))
    dip_headers.append('dip_'+str(i))

sip = []
for ip in edited_df['sip']:
    sip.append(map(int,correct_ip(ip).split('.')))
#     sip.append(correct_ip(ip).split('.'))

dip = []
for ip in edited_df['dip']:
    dip.append(map(int,correct_ip(ip).split('.')))
#     dip.append(correct_ip(ip).split('.'))


        
sport = []
for port in edited_df['sport']:
    sport.append(int(port))
    
dport = []
for port in edited_df['dport']:
    dport.append(int(port))
# print(len(sip[0]))
# print(len(dip[0]))
# print(len(dport[0]))
# print(len(sport[0]))
8
sip_df = pd.DataFrame(sip,columns=sip_headers)
dip_df = pd.DataFrame(dip,columns=dip_headers)
sport_df = pd.DataFrame(sport,columns=['sport'])
dport_df = pd.DataFrame(dport,columns=['dport'])


result = pd.concat([sip_df, dip_df, sport_df, dport_df, one_hot], axis=1, sort=False)
result.head()
# print(df['protocol'].nunique())

data read
data sorted
data : 213446
dataset created
                     stime                    etime             sip  sport  \
0  2017-05-09 10:36:34.876  2017-05-09 11:05:37.894  192.168.80.102   5353   
1  2017-05-09 10:38:11.077  2017-05-09 11:06:13.402  192.168.110.60  34056   
2  2017-05-09 10:38:11.077  2017-05-09 11:06:13.402  192.168.110.60  34056   
3  2017-05-09 10:38:11.077  2017-05-09 11:06:13.402  192.168.110.60  34056   
4  2017-05-09 10:38:11.078  2017-05-09 11:06:13.402  192.168.110.60  34056   

       sipint                 mac osname osversion fingerprint  \
0  3232256102  20:cf:30:8b:6f:17:                                
1  3232263740  00:26:18:f0:62:08:                                
2  3232263740  00:26:18:f0:62:08:                                
3  3232263740  00:26:18:f0:62:08:                                
4  3232263740  00:26:18:f0:62:08:                                

               dip     ...     ruflags entropy rentropy tos rtos application  \
0 

,sip_0,sip_1,sip_2,sip_3,dip_0,dip_1,dip_2,dip_3,sport,dport,protocol_1,protocol_139,protocol_17,protocol_2,protocol_41,protocol_47,protocol_6
0,192,168,80,102,224,0,0,251,5353,5353,0,0,1,0,0,0,0
1,192,168,110,60,171,122,234,240,34056,3395,0,0,1,0,0,0,0
2,192,168,110,60,182,132,115,102,34056,3395,0,0,1,0,0,0,0
3,192,168,110,60,182,40,209,121,34056,3395,0,0,1,0,0,0,0
4,192,168,110,60,124,232,60,77,34056,3395,0,0,1,0,0,0,0


In [33]:
from keras.datasets import mnist
import numpy as np

(x_train, y_train), (x_test, y_test) = mnist. load_data()

def cal_Euclidean_distance(a,b):
    distance = float(0)
    for parameter in range(len(a)):
        distance += (a[parameter]-b[parameter])**(2)
    return distance**(float(1/2))


def inequality_distance(a,b):
    x = (a-b)**2
    if x == 1:
        return 1
    return min(x-1,0)*x/(x-1) + max(x-1,0)*(2*x-1)/(x*(x-1))
                
# print(inequality_distance(4,1000))

def cal_my_distance(a,b):
    sip0_loss = 4 * inequality_distance(a[0],b[0])
    sip1_loss = 3 * inequality_distance(a[1],b[1])
    sip2_loss = 2 * inequality_distance(a[2],b[2])
    sip3_loss = inequality_distance(a[3],b[3])

    sip_loss = sip0_loss + sip1_loss + sip2_loss + sip3_loss 

    dip0_loss = 16 * inequality_distance(a[4],b[4])
    dip1_loss = 9 * inequality_distance(a[5],b[5])
    dip2_loss = 4 * inequality_distance(a[6],b[6])
    dip3_loss = inequality_distance(a[7],b[7])

    dip_loss = dip0_loss + dip1_loss + dip2_loss + dip3_loss 

#     x = K.square(y_pred - y_true)
#     sport_loss = K.mean(K.min(x-1,0)*x/(x-1) + K.max(x-1,0)*(2*x -1)/2/x/(x-1),axis=-1)
    sport_loss = inequality_distance(a[8],b[8])
    dport_loss = 4 * inequality_distance(a[9],b[9])
#     protocol_loss = K.mean(K.square(y_true[10:17]-y_pred[10:17]),axis=-1)
#     print(protocol_loss.shape)
    # Full loss
    model_loss = sip_loss + dip_loss + sport_loss + dport_loss
    return model_loss
# def find_neighbors(current_point):
#     for i in range(x_train.shape[0]):
#         if (flags[i]):
            
            
            
# print(cal_Euclidean_distance([0,0],[3,4]))


# df2 = x_train.assign(address = [].zfill(x_train.shape[0])) 

flags = np.ones((x_train.shape[0],1))
# print(x_train.shape)
# print(flags.shape)



In [34]:
import sklearn
from sklearn.neighbors import NearestNeighbors
import numpy as np

nbrs = NearestNeighbors(n_neighbors=4, algorithm='ball_tree', metric=cal_my_distance)
nbrs.fit(result)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars


NearestNeighbors(algorithm='ball_tree', leaf_size=30,
         metric=<function cal_my_distance at 0x1a570102f0>,
         metric_params=None, n_jobs=None, n_neighbors=4, p=2, radius=1.0)

In [35]:
nbrs

NearestNeighbors(algorithm='ball_tree', leaf_size=30,
         metric=<function cal_my_distance at 0x1a570102f0>,
         metric_params=None, n_jobs=None, n_neighbors=4, p=2, radius=1.0)